In [1]:
import os
import pandas as pd
import numpy as np
import sqlite3
%reload_ext sql

## Setup sqlite3

1. Download the sqlite tools precompiled binaries for your OS from: https://www.sqlite.org/download.html
2. Create an sqlite folder somewhere like in your ProgramFiles or User Home directory and unpack the tools into the file
3. Add the file location to your PATH variables

helpful resource: https://www.sqlitetutorial.net/sqlite-python/creating-database/

for my setup, simply did the above, and added a db folder to store the local dbs

## Creating Database tables

In [3]:
db = sqlite3.connect(r"C:\sqlite\db\CS513.db")

In [4]:
create_table_rest = """
CREATE TABLE IF NOT EXISTS restaurant(
license BIGINT PRIMARY KEY,
dba_name VARCHAR,
aka_name VARCHAR,
facility_type VARCHAR,
risk VARCHAR,
address VARCHAR,
city VARCHAR,
state VARCHAR,
zip INT,
lat REAL,
long REAL
);
"""

In [5]:
create_table_violation = """
CREATE TABLE IF NOT EXISTS violation(
violation_num INT PRIMARY KEY,
desc VARCHAR
);
"""

In [6]:
create_table_insp = """
CREATE TABLE IF NOT EXISTS inspection(
inspection_id BIGINT PRIMARY KEY,
license BIGINT,
insp_date DATE,
insp_type VARCHAR,
results VARCHAR,
violation_num INT,
comments VARCHAR,
FOREIGN KEY (license) REFERENCES restaurant (license),
FOREIGN KEY (violation_num) REFERENCES violation (violation_num)
);
"""

In [46]:
db.execute(create_table_rest)

In [47]:
db.execute(create_table_violation)

In [48]:
db.execute(create_table_insp)

## Adding Data to violation table

In [9]:
#Reading in Parsed Violations and formatting for insert
vio_df = pd.read_csv("Violations_Parsed.csv")
vio_df = vio_df[['Violation #', 'Violation Desc']]
vio_df = vio_df.drop_duplicates(subset='Violation #')
vio_df = vio_df.sort_values(by='Violation #')

#Renaming to match existing db table's data names
vio_df.columns = ['violation_num','desc']

#Using Pandas to append data to existing db table
vio_df.to_sql('violation',con=db, if_exists='append',index=False)